# Tutorial Part 28: Retrosynthesis with AiZynthFinder

This tutorial demonstrates using the retrosynthesis module in DeepChem that is using the package `aizynthfinder` as the engine.

We will install `aizynthfinder` alongside `deepchem` and then download public policies and stocks from Figshare.
Then, we will see how we can predict the synthesis route of the drug *amenamevir*

## Colab

This tutorial and the rest in this sequence can be done in Google Colab. If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/29_Retrosynthesis_with_AiZynthFinder.ipynb)

## Setup

To run DeepChem within Colab, you'll need to run the following cell of installation commands. This will take about 5 minutes to run to completion and install your environment.

In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

In [ ]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

### AiZynthFinder installation

With `deepchem` installed, we can proceed with installing `aizynthfinder` and `graphviz`, which will be used to make route images.

This you need to do even if you have setup `deepchem` locally. On colab it is not necessary to install `graphviz`,
so we comment that line.

In [ ]:
#!conda install graphviz -y
!python -m pip install https://github.com/MolecularAI/aizynthfinder/archive/v2.4.0.tar.gz

Then we will download publicly available policies and stocks.

In [ ]:
!mkdir --parents data
!download_public_data data

## Retrosynthesis

We will start with setting up the retrosynthesis engine. This is a wrapper around `aizynthfinder` with a simplified
interface.

We will initialize the engine by providing it with the paths to the two expansion policy files, and with the path to the stock files.
The first expansion policy file is a Keras checkpoint file for a template recommendation engine,
whereas the second expansion policy file is the list of the actual templates.

In [ ]:
from deepchem.applications.synthesis import RetrosynthesisEngine
expansion_policy_paths = (
    "data/uspto_model.hdf5",
    "data/uspto_templates.hdf5"
)
stock_path = "data/zinc_stock.hdf5"
engine = RetrosynthesisEngine(expansion_policy_paths, stock_path)

Now we are ready to predict the synthesis route for a molecule. Here, we will use the
drug *amenamevir* as an example. The analysis will take up to 2 min.

In [ ]:
routes = engine.find_routes("Cc1cccc(c1N(CC(=O)Nc2ccc(cc2)c3ncon3)C(=O)C4CCS(=O)(=O)CC4)C")

Now we have the predicted routes in the `routes` variable, which is a list of dictionaries.
This we can use to dump information about the predictions.

In [ ]:
for route in routes:
    display(route["image"])
    print(f"MCTS score: {route['scores']['state score']:.2f}")
    print(f"Pre-cursors in stock: {','.join(route['in_stock'])}")
    print(f"Pre-cursors not in stock: {','.join(route['not_in_stock'])}")

This just show the simplest interface, and there is a lot more things that can be configured.

We can for instance add a filter policy to our engine, that will improve the accuracy of the predictions

In [ ]:
engine.update_filter_policy("data/uspto_filter_model.hdf5")

and we can increase both the number of iterations the search time:

In [ ]:
engine.finder_settings = {
    "iteration_limit": 500,
    "time_limit": 600,
}

# Bibliography:

Genheden S, Thakkar A, Chadimova V, et al (2020) AiZynthFinder: a fast, robust and flexible open-source software for retrosynthetic planning. J. Cheminf. https://doi.org/10.1186/s13321-020-00472-1

# Congratulations! Time to join the Community!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:

## Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.

## Join the DeepChem Gitter
The DeepChem [Gitter](https://gitter.im/deepchem/Lobby) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!